#### Prepare Cellpose training dataset  

30 training images from the HCC1143 series of experiments. Early and late Untreated plus late high dose from each drug. For nuclear segmentation, apply the red channel as a mask to the phase, then combine them to get the red channel "textured" by the phase images. Use red and phase as separate channels for cytoplasmic segmentation.  Use registered intensity images and prefiltered masks. Masks must have the same name as the intensity images with a suffix added before the file type. 

In [ ]:
import numpy as np
import numpy.ma as ma
import os, glob, tifffile
from skimage import io, filters, exposure

In [ ]:
def transfer_file(data_path, plateID, model_name, well, field_num, time_slice):
    
    #get the cropped rp image
    filename = os.path.join(data_path, plateID,"Analysis","registered_stacks",plateID+"_RP_"+well+"_"+str(field_num)+"_reg_stack.tif")
    print("preparing "+filename+" for training set")
    image = io.imread(filename)[int(time_slice)]
    
    #add the cropped rp image to the training set of images
    rp_out_filename = os.path.join(data_path,"cellpose_training","train",plateID+"_"+well+"_"+str(field_num)+"_"+time_slice+"_rp_img.tif")
    tifffile.imwrite(rp_out_filename, image)

hist, bins_center = exposure.histogram(image)

plt.figure(figsize=(18, 8))
plt.subplot(131)
plt.imshow(image, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(132)
plt.imshow(nuc_mask, cmap='gray', interpolation='nearest')
plt.axis('off')
plt.subplot(133)
plt.imshow(phase_image_masked, cmap = 'gray', interpolation = 'nearest')
plt.tight_layout()

In [ ]:
data_path = "/home/exacloud/gscratch/HeiserLab/images/"
plateID_list = ("HC00701", "HC00801", "HC00901", "HC01001", "HC01301", "HC01401")
#plateID_list = (["HC01401"])
model_name = "Ctn"
well_list = ("A1", "A1","D2", "D4", "D6")
time_slice_list = ("004", "188", "188", "188", "188")
#well_list = (["A1"])
field_num = 1


In [ ]:
res = list(map(transfer_file, [data_path]*len(well_list)*len(plateID_list),
               [element for element in plateID_list for i in range(len(well_list))],[model_name]*len(well_list)*len(plateID_list),
               well_list*len(plateID_list),
               [field_num]*len(well_list)*len(plateID_list),
               time_slice_list*len(plateID_list)))
